## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-05-20-52-11 +0000,wapo,Live updates: Venezuelan leader Maduro pleads ...,https://www.washingtonpost.com/world/2026/01/0...
1,2026-01-05-20-52-11 +0000,wapo,State Dept. makes plans that would allow Trump...,https://www.washingtonpost.com
2,2026-01-05-20-46-27 +0000,nyt,Venezuela Live Updates: Maduro Tells U.S. Judg...,https://www.nytimes.com/live/2026/01/05/world/...
3,2026-01-05-20-44-00 +0000,wsj,"GM, Toyota Hint at Tough Year Ahead for U.S. A...",https://www.wsj.com/business/autos/auto-car-sa...
4,2026-01-05-20-43-48 +0000,startribune,"Live: Vikings’ 2026 opponents, first-round dra...",https://www.startribune.com/minnesota-vikings-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2378/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,venezuela,67
4,maduro,51
13,trump,47
170,new,20
59,oil,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
239,2026-01-05-00-43-48 +0000,cbc,"After seizing Maduro, Trump wants Venezuela's ...",https://www.cbc.ca/news/world/trump-venezuela-...,211
193,2026-01-05-10-00-26 +0000,wapo,"What to know about Maduro, his capture and Tru...",https://www.washingtonpost.com/world/2026/01/0...,180
196,2026-01-05-09-20-28 +0000,wapo,Trump says U.S. is ‘in charge’ in Venezuela af...,https://www.washingtonpost.com/world/2026/01/0...,178
173,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...,164
182,2026-01-05-10-47-00 +0000,wsj,Oil Prices Fall as U.S. Capture of Venezuela’s...,https://www.wsj.com/business/energy-oil/asian-...,159


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
239,211,2026-01-05-00-43-48 +0000,cbc,"After seizing Maduro, Trump wants Venezuela's ...",https://www.cbc.ca/news/world/trump-venezuela-...
206,66,2026-01-05-07-55-46 +0000,nypost,Gov. Tim Walz expected to drop bid for re-elec...,https://nypost.com/2026/01/05/us-news/gov-tim-...
56,57,2026-01-05-19-11-00 +0000,wsj,U.S. companies will benefit under a new global...,https://www.wsj.com/politics/policy/oecd-corpo...
87,57,2026-01-05-18-04-32 +0000,nypost,Toppled Venezuelan dictator NicoláMaduro decla...,https://nypost.com/2026/01/05/us-news/toppled-...
173,40,2026-01-05-12-00-00 +0000,wsj,Secretary of State Marco Rubio was there with ...,https://www.wsj.com/politics/elections/in-vene...
217,37,2026-01-05-04-29-58 +0000,nypost,Trump renews calls to take over Greenland — as...,https://nypost.com/2026/01/04/world-news/whos-...
143,33,2026-01-05-15-15-57 +0000,nypost,"Sen. Mark Kelly at risk of demotion, cut in mi...",https://nypost.com/2026/01/05/us-news/sen-mark...
38,33,2026-01-05-19-52-57 +0000,nyt,Thousands in Berlin Go Days Without Power and ...,https://www.nytimes.com/2026/01/05/world/europ...
166,30,2026-01-05-12-34-21 +0000,nyt,"Delcy Rodríguez, Venezuela’s New Leader, Faces...",https://www.nytimes.com/2026/01/04/world/ameri...
230,30,2026-01-05-02-00-00 +0000,wsj,Trump’s Venezuela incursion has injected a vol...,https://www.wsj.com/politics/policy/trump-is-t...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
